# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 60 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/21 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.14312


extracting tarball to tmp_2209.14312...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.14314


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.14312/natnotes.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

extracting tarball to tmp_2209.14844...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.14866


extracting tarball to tmp_2209.14866...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.14897


/tmp/ipykernel_2259/4030337529.py:34: LatexWarning: 2209.14897 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14396-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14396) | **TESS spots a mini-neptune interior to a hot saturn in the TOI-2000  system**  |
|| <mark>Lizhou Sha</mark>, et al. -- incl., <mark>Melissa J. Hobson</mark>, <mark>Carl Ziegler</mark>, <mark>Pascal Torres-Miranda</mark>, <mark>Trifon Trifonov</mark>, <mark>Colin Littlefield</mark>, <mark>Jan Eberhardt</mark>, <mark>Richard P. Schwarz</mark>, <mark>Thomas Henning</mark>, <mark>Jack J. Lissauer</mark> |
|*Appeared on*| *2022-09-30*|
|*Comments*| *20 pages, 7 figures, 13 tables; submitted to MNRAS; CSV tables available as ancillary files; posterior samples available from Zenodo at this https URL*|
|**Abstract**| Hot jupiters (P < 10 d, M > 60 $\mathrm{M}_\oplus$) are almost always found alone around their stars, but four out of hundreds known have inner companion planets. These rare companions allow us to constrain the hot jupiter's formation history by ruling out high-eccentricity tidal migration. Less is known about inner companions to hot Saturn-mass planets. We report here the discovery of the TOI-2000 system, which features a hot Saturn-mass planet with a smaller inner companion. The mini-neptune TOI-2000 b ($2.64^{+0.11}_{-0.12} \,\mathrm{R}_\oplus$, $10.3 \pm 2.2 \,\mathrm{M}_\oplus$) is in a 3.10-day orbit, and the hot saturn TOI-2000 c ($7.97 \pm 0.12 \,\mathrm{R}_\oplus$, $75.7 \pm 3.8 \,\mathrm{M}_\oplus$) is in a 9.13-day orbit. Both planets transit their host star TOI-2000 (TIC 371188886, V = 10.98, TESS magnitude = 10.36), a metal-rich ([Fe/H] = $0.438^{+0.041}_{-0.042}$) G dwarf 174 pc away. TESS observed the two planets in sectors 9-11 and 36-38, and we followed up with ground-based photometry, spectroscopy, and speckle imaging. Radial velocities from HARPS allowed us to confirm both planets by direct mass measurement. In addition, we demonstrate constraining planetary and stellar parameters with MIST stellar evolutionary tracks through Hamiltonian Monte Carlo under the PyMC framework, achieving higher sampling efficiency and shorter run time compared to traditional Markov chain Monte Carlo. Having the brightest host star in the V band among similar systems, TOI-2000 b and c are superb candidates for atmospheric characterization by the JWST, which can potentially distinguish whether they formed together or TOI-2000 c swept along material during migration to form TOI-2000 b. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14410-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14410) | **Towards measuring supermassive black hole masses with interferometric  observations of the dust continuum**  |
|| GRAVITY Collaboration, et al. -- incl., <mark>R. Davies</mark>, <mark>N. M. Förster Schreiber</mark> |
|*Appeared on*| *2022-09-30*|
|*Comments*| *Accepted for publication in A&A; 11 pages, 5 figures, 3 tables*|
|**Abstract**| This work focuses on active galactic nuclei (AGNs), and the relation between the sizes of the hot dust continuum and the broad-line region (BLR). We find that the continuum size measured using optical/near-infrared interferometry (OI) is roughly twice that measured by reverberation mapping (RM). Both OI and RM continuum sizes show a tight relation with the H$\beta$ BLR size with only an intrinsic scatter of 0.25 dex. The masses of supermassive black holes (BHs) can hence be simply derived from a dust size in combination with a broad line width and virial factor. Since the primary uncertainty of these BH masses comes from the virial factor, the accuracy of the continuum-based BH masses is close to those based on the RM measurement of the broad emission line. Moreover, the necessary continuum measurements can be obtained on a much shorter timescale than those required monitoring for RM, and are also more time efficient than those needed to resolve the BLR with OI. The primary goal of this work is to demonstrate measuring the BH mass based on the dust continuum size with our first calibration of the $R_\mathrm{BLR}$-$R_\mathrm{d}$ relation. The current limitation and caveats are discussed in detail. Future GRAVITY observations are expected to improve the continuum-based method and have the potential to measure BH masses for a large sample of AGNs in the low-redshift Universe. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14312-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14312) | **Bridging the Gap between Cosmic Dawn and Reionization favors Faint  Galaxies-dominated Models**  |
|| Ankita Bera, et al. -- incl., <mark>Aaron Smith</mark> |
|*Appeared on*| *2022-09-30*|
|*Comments*| *12 pages, 5 figures, submitted to ApJ, comments are welcome*|
|**Abstract**| Current standard astrophysical models struggle to explain the tentative detection of the 21 cm absorption trough centered at $z\sim17$ measured by the EDGES low-band antenna. However, it has been shown that the EDGES results are consistent with an extrapolation of a declining UV luminosity density, following a simple power-law of deep Hubble Space Telescope observations of $4 < z < 9$ galaxies. We here explore the conditions by which the EDGES detection is consistent with current reionization and post-reionization observations, including the volume-averaged neutral hydrogen fraction of the intergalactic medium at $z\sim6-8$, the optical depth to the cosmic microwave background, and the integrated ionizing emissivity at $z\sim5$. By coupling a physically motivated source model derived from radiative transfer hydrodynamic simulations of reionization to a Markov Chain Monte Carlo sampler, we find that high contribution from low-mass halos along with high photon escape fractions are required to simultaneously reproduce the high-redshift (cosmic dawn) and low-redshift (reionization) existing constraints. Low-mass faint-galaxies dominated models produce a flatter emissivity evolution that results in an earlier onset of reionization with gradual and longer duration, and higher optical depth. Our results provide insights on the role of faint and bright galaxies during cosmic reionization, which can be tested by upcoming surveys with the James Webb Space Telescope. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14830-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14830) | **An old warm Jupiter orbiting the metal-poor G-dwarf TOI-5542**  |
|| Nolan Grieves, et al. -- incl., <mark>Beth A. Henderson</mark>, <mark>Christopher E. Henze</mark> |
|*Appeared on*| *2022-09-30*|
|*Comments*| *16 pages, 12 figures, Accepted by Astronomy & Astrophysics September 19, 2022*|
|**Abstract**| We report the discovery of a 1.32$^{+0.10}_{-0.10}$ $\mathrm{M_{\rm Jup}}$ planet orbiting on a 75.12 day period around the G3V $10.8^{+2.1}_{-3.6}$ Gyr old star TOI-5542 (TIC 466206508; TYC 9086-1210-1). The planet was first detected by the Transiting Exoplanet Survey Satellite (TESS) as a single transit event in TESS Sector 13. A second transit was observed 376 days later in TESS Sector 27. The planetary nature of the object has been confirmed by ground-based spectroscopic and radial velocity observations from the CORALIE and HARPS spectrographs. A third transit event was detected by the ground-based facilities NGTS, EulerCam, and SAAO. We find the planet has a radius of 1.009$^{+0.036}_{-0.035}$ $\mathrm{R_{\rm Jup}}$ and an insolation of 9.6$^{+0.9}_{-0.8}$ $S_{\oplus}$, along with a circular orbit that most likely formed via disk migration or in situ formation, rather than high-eccentricity migration mechanisms. Our analysis of the HARPS spectra yields a host star metallicity of [Fe/H] = $-$0.21$\pm$0.08, which does not follow the traditional trend of high host star metallicity for giant planets and does not bolster studies suggesting a difference among low- and high-mass giant planet host star metallicities. Additionally, when analyzing a sample of 216 well-characterized giant planets, we find that both high masses (4 $\mathrm{M_{\rm Jup}}$ $<M_{p}<$ 13 $\mathrm{M_{\rm Jup}}$) and low masses (0.5 $\mathrm{M_{\rm Jup}}$ $<M_{p}<$ 4 $\mathrm{M_{\rm Jup}}$), as well as both both warm (P $>$ 10 days) and hot (P $<$ 10 days) giant planets are preferentially located around metal-rich stars (mean [Fe/H] $>$ 0.1). TOI-5542b is one of the oldest known warm Jupiters and it is cool enough to be unaffected by inflation due to stellar incident flux, making it a valuable contribution in the context of planetary composition and formation studies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14844-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14844) | **First measurements and upgrade plans of the MAGIC intensity  interferometer**  |
|| Juan Cortina, et al. -- incl., <mark>R. Walter</mark> |
|*Appeared on*| *2022-09-30*|
|*Comments*| *15 pages, 9 figures, 1 table, SPIE Astronomical Telescopes + Instrumentation 2022*|
|**Abstract**| The two MAGIC 17-m diameter Imaging Atmospheric Cherenkov Telescopes have been equipped to work also as an intensity interferometer with a deadtime-free, 4-channel, GPU-based, real-time correlator. Operating with baselines between approx. 40 and 90 m the MAGIC interferometer is able to measure stellar diameters of 0.5-1 mas in the 400-440 nm wavelength range with a sensitivity roughly 10 times better than that achieved in the 1970s by the Narrabri Stellar Intensity Interferometer. Besides, active mirror control allows to split the primary mirrors into sub-mirrors. This allows to make simultaneous calibration measurements of the zero-baseline correlation or to simultaneously collect six baselines below 17 m with almost arbitrary orientation, corresponding to angular scales of approx. 1-50 mas. We plan to perform test observations adding the nearby Cherenkov Telescope Array (CTA) LST-1 23 m diameter telescope by next year. All three telescope pairs will be correlated simultaneously. Adding LST-1 is expected to increase the sensitivity by at least 1 mag and significantly improve the u-v plane coverage. If successful, the proposed correlator setup is scalable enough to be implemented to the full CTA arrays. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14866-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14866) | **A Highly Magnified Star at Redshift 6.2**  |
|| Brian Welch, et al. -- incl., <mark>Guillaume Mahler</mark>, <mark>Soniya Sharma</mark> |
|*Appeared on*| *2022-09-30*|
|*Comments*| *50 pages, 11 figures (3 main text, 8 extended data). Published in Nature*|
|**Abstract**| Galaxy clusters magnify background objects through strong gravitational lensing. Typical magnifications for lensed galaxies are factors of a few but can also be as high as tens or hundreds, stretching galaxies into giant arcs. Individual stars can attain even higher magnifications given fortuitous alignment with the lensing cluster. Recently, several individual stars at redshift $z \sim 1 - 1.5$ have been discovered, magnified by factors of thousands, temporarily boosted by microlensing. Here we report observations of a more distant and persistent magnified star at redshift $z_{\rm phot} = 6.2 \pm 0.1$, 900 Myr after the Big Bang. This star is magnified by a factor of thousands by the foreground galaxy cluster lens WHL0137--08 ($z = 0.566$), as estimated by four independent lens models. Unlike previous lensed stars, the magnification and observed brightness (AB mag 27.2) have remained roughly constant over 3.5 years of imaging and follow-up. The delensed absolute UV magnitude $M_{UV} = -10 \pm 2$ is consistent with a star of mass $M > 50 M_{\odot}$. Confirmation and spectral classification are forthcoming from approved observations with the James Webb Space Telescope |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14629-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14629) | **DeGaPe 35: Amateur discovery of a new southern symbiotic star**  |
|| Thomas Petit, et al. -- incl., <mark>Lionel Mulato</mark> |
|*Appeared on*| *2022-09-30*|
|*Comments*| *accepted for publication in New Astronomy; 7 pages, 5 figures, 1 table*|
|**Abstract**| In this work, we present the discovery and characterization of a new southern S-type symbiotic star, DeGaPe 35. We have obtained the low-resolution spectroscopic observations and supplemented them with photometry from Gaia DR3 and other surveys. The optical spectra of this target show prominent emission lines, including highly ionized [Fe VII] and O VI lines. The cool component of this symbiotic binary is an M4-5 giant with effective temperature ~ 3 380 - 3 470 K and luminosity ~ 3 000 L$_\odot$ (for the adopted distance of 3 kpc). The hot component is a shell-burning white dwarf. The photometric observations of the Gaia satellite, published recently in the Gaia DR3 suggested the variability with the period of about 700 - 800 days that we tentatively attributed to the orbital motion of the binary. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.14897-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.14897) | **Thermodynamics of the inner heliosheath**  |
|| <mark>G. Livadiotis</mark>, et al. |
|*Appeared on*| *2022-09-30*|
|*Comments*| *34 pages, 18 figures*|
|**Abstract**| We derive annual skymaps of the proton temperature in the inner heliosheath (IHS), and track their temporal evolution over the years from 2009 to 2016 of Interstellar Boundary Explorer observations. Other associated thermodynamic parameters also determined are the density, kappa, that is, the parameter that characterizes kappa distributions, temperature rate, polytropic index, and entropy. We exploit the theory of kappa distributions and their connection with polytropes, to (i) express a new polytropic quantity {\Pi} that remains invariant along streamlines where temperature and density may vary, (ii) parameterize the proton flux in terms of the {\Pi} invariant and kappa, and (iii) derive the temperature and density, respectively, from the slope and intercept of the linear relationship between kappa and logarithm of {\Pi}. We find the following thermodynamic characteristics: (1) Temperature sky-maps and histograms shifted to their lowest values in 2012 and their highest in 2015; (2) Temperature negatively correlated with density, reflecting the subisothermal polytropic behavior; (3) Temperature positively correlated with kappa, revealing characteristics of the mechanism responsible for generating kappa distributions; (4) Processes in IHS are sub-isothermal tending toward isobaric, consistent with previously published results; (5) Linear relationship between kappa and polytropic indices, revealing characteristics of the particle potential energy; and (6) Entropy positively correlated with polytropic index, aligned with the underlying theory that entropy increases towards the isothermal state where the kappa distribution reduces to the Maxwell Boltzmann description. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.14396.md
    + _build/html/tmp_2209.14396/./figures/family_portrait.png
    + _build/html/tmp_2209.14396/./figures/toi_2000_lc.png
    + _build/html/tmp_2209.14396/./figures/toi_2000_mass_radius.png
exported in  _build/html/2209.14410.md
    + _build/html/tmp_2209.14410/./fig2.png
    + _build/html/tmp_2209.14410/./fig1.png
    + _build/html/tmp_2209.14410/./fig3.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\plusminus}[3]{#1^{+#2}_{-#3}}$
$\newcommand{\circa}{\ensuremath{{\sim}\mspace{2mu}}}$
$\newcommand{\ut}{\textsc{ut}}$
$\newcommand{\tablenotemark}[1]{\textsuperscript{\textit{#1}}}$
$\newcommand{\bjdtdb}{\ensuremath{\mathrm{BJD}_\text{TDB}}}$
$\newcommand{\feh}{\ensuremath{[\text{Fe}/\text{H}]}}$
$\newcommand{\logg}{\ensuremath{\log g}}$
$\newcommand{\tempeff}{\ensuremath{T_{\text{eff}}}}$
$\newcommand{\tempeq}{\ensuremath{T_{\text{eq}}}}$
$\newcommand{\vsini}{\ensuremath{v \sin i}}$
$\newcommand{\normaldist}{\mathcal{N}}$
$\newcommand{\uniformdist}{\mathcal{U}}$
$\newcommand{\gaiaG}{\ensuremath{G}}$
$\newcommand{\gaiaBP}{\ensuremath{G_\mathrm{BP}}}$
$\newcommand{\gaiaRP}{\ensuremath{G_\mathrm{RP}}}$
$\newcommand{\filterB}{\ensuremath{B}}$
$\newcommand{\filterIc}{\ensuremath{I_\mathrm{c}}}$
$\newcommand{\filterJ}{\ensuremath{J}}$
$\newcommand{\filterH}{\ensuremath{H}}$
$\newcommand{\filterK}{\ensuremath{K_\mathrm{s}}}$
$\newcommand{\filterR}{\ensuremath{R}}$
$\newcommand{\filterRc}{\ensuremath{R_\mathrm{c}}}$
$\newcommand{\filterV}{\ensuremath{V}}$
$\newcommand{\filterWone}{\ensuremath{W1}}$
$\newcommand{\filterWtwo}{\ensuremath{W2}}$
$\newcommand{\filterWthree}{\ensuremath{W3}}$
$\newcommand{\filterWfour}{\ensuremath{W4}}$
$\newcommand{\filtergp}{\ensuremath{g'}}$
$\newcommand{\filterip}{\ensuremath{i'}}$
$\newcommand{\filterrp}{\ensuremath{r'}}$
$\newcommand{\filterzs}{\ensuremath{z_\mathrm{s}}}$
$\newcommand{\ares}{\textsc{ares}}$
$\newcommand{\arviz}{\textsc{arviz}}$
$\newcommand{\astroimagej}{\textsc{AstroImageJ}}$
$\newcommand{\batman}{\textsc{batman}}$
$\newcommand{\celeritetwo}{\textsc{celerite2}}$
$\newcommand{\emcee}{\textsc{emcee}}$
$\newcommand{\exofast}{\textsc{exofast}}$
$\newcommand{\exoplanetpy}{\textsc{exoplanet}}$
$\newcommand{\isochrones}{\textsc{isochrones}}$
$\newcommand{\mist}{MIST}$
$\newcommand{\molusc}{\textsc{molusc}}$
$\newcommand{\moog}{\textsc{moog}}$
$\newcommand{\pymc}{\textsc{PyMC}}$
$\newcommand{\qlp}{\textsc{qlp}}$
$\newcommand{\radvel}{\textsc{radvel}}$
$\newcommand{\sklearn}{\textsc{Scikit-learn}}$
$\newcommand{\triceratops}{\textsc{triceratops}}$
$\newcommand{\vartools}{\textsc{Vartools}}$
$\newcommand{\planet}[2]{#1\thinspace #2}$
$\newcommand{\toitwothousand}{TOI-2000}$
$\newcommand{\toitwothousandtic}{371188886}$
$\newcommand{\planetinner}{\planet{\toitwothousand}{b}}$
$\newcommand{\planetouter}{\planet{\toitwothousand}{c}}$
$\newcommand{\waspfortyseven}{WASP-47}$
$\newcommand{\waspfortysevenhj}{\planet{\waspfortyseven}{b}}$
$\newcommand{\waspfortyseveninner}{\planet{\waspfortyseven}{e}}$
$\newcommand{\waspfortysevenneptune}{\planet{\waspfortyseven}{d}}$
$\newcommand{\keplerseventhirty}{Kepler-730}$
$\newcommand{\keplerseventhirtyhj}{\planet{Kepler-730}{b}}$
$\newcommand{\keplerseventhirtyinner}{\planet{Kepler-730}{c}}$
$\newcommand{\toieleventhirty}{TOI-1130}$
$\newcommand{\toieleventhirtyinner}{\planet{\toieleventhirty}{b}}$
$\newcommand{\toieleventhirtyhj}{\planet{\toieleventhirty}{c}}$
$\newcommand{\keplerthirty}{Kepler-30}$
$\newcommand{\keplerthirtyb}{\planet{\keplerthirty}{b}}$
$\newcommand{\keplereightynine}{KOI-94}$
$\newcommand{\keplereightynineb}{\planet{\keplereightynine}{b}}$
$\newcommand{\wasponethirtytwo}{WASP-132}$
$\newcommand{\wasponethirtytwohj}{\planet{WASP-132}{b}}$
$\newcommand{\wasponethirtytwoinner}{\planet{WASP-132}{c}}$
$\newcommand{\astep}{ASTEP}$
$\newcommand{\chiron}{CHI\-RON}$
$\newcommand{\edrthree}{EDR3}$
$\newcommand{\feros}{FEROS}$
$\newcommand{\gaia}{\emph{Gaia}}$
$\newcommand{\geminisouth}{Gemini South}$
$\newcommand{\harps}{HARPS}$
$\newcommand{\jwst}{\emph{JWST}}$
$\newcommand{\kepler}{\emph{Kepler}}$
$\newcommand{\ktwo}{\emph{K2}}$
$\newcommand{\lcogt}{LCOGT}$
$\newcommand{\pest}{PEST}$
$\newcommand{\soar}{SOAR}$
$\newcommand{\tess}{\emph{TESS}}$
$\newcommand{\twomass}{2MASS}$
$\newcommand{\wise}{\emph{WISE}}$
$\newcommand{\sysParamPeriodSubZero}{9.1270553}$
$\newcommand{\sysParamPeriodSubZeroUnc}{\pm 0.0000073}$
$\newcommand{\sysParamPeriodSubOne}{3.098331}$
$\newcommand{\sysParamPeriodSubOneUnc}{_{-0.000019}^{+0.000021}}$
$\newcommand{\sysParamTzeroSubZero}{2459110.06589}$
$\newcommand{\sysParamTzeroSubZeroUnc}{\pm 0.00027}$
$\newcommand{\sysParamTzeroSubOne}{2458901.7192}$
$\newcommand{\sysParamTzeroSubOneUnc}{_{-0.0020}^{+0.0021}}$
$\newcommand{\sysParamRpSubZero}{0.06564}$
$\newcommand{\sysParamRpSubZeroUnc}{\pm 0.00064}$
$\newcommand{\sysParamRpSubOne}{0.02178}$
$\newcommand{\sysParamRpSubOneUnc}{_{-0.00096}^{+0.00089}}$
$\newcommand{\sysParamBSubZero}{0.618}$
$\newcommand{\sysParamBSubZeroUnc}{_{-0.042}^{+0.037}}$
$\newcommand{\sysParamBSubOne}{0.763}$
$\newcommand{\sysParamBSubOneUnc}{_{-0.072}^{+0.033}}$
$\newcommand{\sysParamTdurSubZero}{3.90}$
$\newcommand{\sysParamTdurSubZeroUnc}{\pm 0.40}$
$\newcommand{\sysParamTdurSubOne}{1.957}$
$\newcommand{\sysParamTdurSubOneUnc}{_{-0.099}^{+0.208}}$
$\newcommand{\sysParamSqrtEccVecZeroSubZero}{-0.213}$
$\newcommand{\sysParamSqrtEccVecZeroSubZeroUnc}{_{-0.046}^{+0.060}}$
$\newcommand{\sysParamSqrtEccVecZeroSubOne}{-0.08}$
$\newcommand{\sysParamSqrtEccVecZeroSubOneUnc}{_{-0.11}^{+0.13}}$
$\newcommand{\sysParamEccSubZero}{0.064}$
$\newcommand{\sysParamEccSubZeroUnc}{_{-0.022}^{+0.023}}$
$\newcommand{\sysParamOmegaSubZero}{-2.43}$
$\newcommand{\sysParamOmegaSubZeroUnc}{_{-0.46}^{+5.19}}$
$\newcommand{\sysParamRPlanetEarthSubZero}{7.97}$
$\newcommand{\sysParamRPlanetEarthSubZeroUnc}{\pm 0.12}$
$\newcommand{\sysParamRPlanetEarthSubOne}{2.64}$
$\newcommand{\sysParamRPlanetEarthSubOneUnc}{_{-0.12}^{+0.11}}$
$\newcommand{\sysParamMPlanetEarthSubZero}{75.7}$
$\newcommand{\sysParamMPlanetEarthSubZeroUnc}{\pm 3.8}$
$\newcommand{\sysParamMPlanetEarthSubOne}{10.3}$
$\newcommand{\sysParamMPlanetEarthSubOneUnc}{\pm 2.2}$
$\newcommand{\sysParamMPlanetSubZero}{(2.27}$
$\newcommand{\sysParamMPlanetSubZeroUnc}{_{-0.12}^{+0.11}) \times 10^{-4}}$
$\newcommand{\sysParamMPlanetSubOne}{(3.09}$
$\newcommand{\sysParamMPlanetSubOneUnc}{_{-0.66}^{+0.67}) \times 10^{-5}}$
$\newcommand{\sysParamRhoPlanetSubZero}{0.822}$
$\newcommand{\sysParamRhoPlanetSubZeroUnc}{_{-0.056}^{+0.058}}$
$\newcommand{\sysParamRhoPlanetSubOne}{3.06}$
$\newcommand{\sysParamRhoPlanetSubOneUnc}{_{-0.72}^{+0.83}}$
$\newcommand{\sysParamKSubZero}{23.8}$
$\newcommand{\sysParamKSubZeroUnc}{\pm 1.1}$
$\newcommand{\sysParamKSubOne}{4.63}$
$\newcommand{\sysParamKSubOneUnc}{_{-0.99}^{+1.00}}$
$\newcommand{\sysParamASubZero}{0.0875}$
$\newcommand{\sysParamASubZeroUnc}{_{-0.0014}^{+0.0016}}$
$\newcommand{\sysParamASubOne}{0.04260}$
$\newcommand{\sysParamASubOneUnc}{_{-0.00067}^{+0.00076}}$
$\newcommand{\sysParamInclSubZero}{87.94}$
$\newcommand{\sysParamInclSubZeroUnc}{_{-0.11}^{+0.13}}$
$\newcommand{\sysParamInclSubOne}{84.73}$
$\newcommand{\sysParamInclSubOneUnc}{_{-0.33}^{+0.49}}$
$\newcommand{\sysParamAorSubZero}{16.92}$
$\newcommand{\sysParamAorSubZeroUnc}{_{-0.33}^{+0.37}}$
$\newcommand{\sysParamAorSubOne}{8.23}$
$\newcommand{\sysParamAorSubOneUnc}{_{-0.16}^{+0.18}}$
$\newcommand{\sysParamMStarZero}{1.074}$
$\newcommand{\sysParamMStarZeroUnc}{_{-0.050}^{+0.059}}$
$\newcommand{\sysParamFehZero}{0.414}$
$\newcommand{\sysParamFehZeroUnc}{_{-0.041}^{+0.040}}$
$\newcommand{\sysParamEep}{380}$
$\newcommand{\sysParamEepUnc}{_{-36}^{+25}}$
$\newcommand{\sysParamMStar}{1.074}$
$\newcommand{\sysParamMStarUnc}{_{-0.050}^{+0.059}}$
$\newcommand{\sysParamRStar}{1.113}$
$\newcommand{\sysParamRStarUnc}{\pm 0.011}$
$\newcommand{\sysParamRhoStar}{1.099}$
$\newcommand{\sysParamRhoStarUnc}{_{-0.063}^{+0.074}}$
$\newcommand{\sysParamLoggStar}{4.376}$
$\newcommand{\sysParamLoggStarUnc}{_{-0.023}^{+0.026}}$
$\newcommand{\sysParamTeff}{5607}$
$\newcommand{\sysParamTeffUnc}{_{-82}^{+86}}$
$\newcommand{\sysParamFeh}{0.438}$
$\newcommand{\sysParamFehUnc}{_{-0.042}^{+0.041}}$
$\newcommand{\sysParamAge}{5.1}$
$\newcommand{\sysParamAgeUnc}{\pm 2.7}$
$\newcommand{\sysParamParallax}{5.760}$
$\newcommand{\sysParamParallaxUnc}{\pm 0.010}$
$\newcommand{\sysParamAv}{0.156}$
$\newcommand{\sysParamAvUnc}{_{-0.072}^{+0.074}}$
$\newcommand{\sysParamMagBol}{4.634}$
$\newcommand{\sysParamMagBolUnc}{_{-0.054}^{+0.052}}$
$\newcommand{\sysParamLStar}{1.103}$
$\newcommand{\sysParamLStarUnc}{_{-0.051}^{+0.056}}$
$\newcommand{\sysParamDistance}{173.60}$
$\newcommand{\sysParamDistanceUnc}{\pm 0.31}$
$\newcommand{\sysParamSedUncScale}{1.30}$
$\newcommand{\sysParamSedUncScaleUnc}{_{-0.22}^{+0.41}}$
$\newcommand{\sysParamIrradiationSubZero}{(1.961}$
$\newcommand{\sysParamIrradiationSubZeroUnc}{_{-0.094}^{+0.101}) \times 10^{8}}$
$\newcommand{\sysParamIrradiationSubOne}{(8.26}$
$\newcommand{\sysParamIrradiationSubOneUnc}{_{-0.40}^{+0.43}) \times 10^{8}}$
$\newcommand{\sysParamUTessSubZero}{0.124}$
$\newcommand{\sysParamUTessSubZeroUnc}{_{-0.090}^{+0.163}}$
$\newcommand{\sysParamUTessSubOne}{0.34}$
$\newcommand{\sysParamUTessSubOneUnc}{_{-0.23}^{+0.16}}$
$\newcommand{\sysParamURcSubZero}{0.88}$
$\newcommand{\sysParamURcSubZeroUnc}{_{-0.28}^{+0.21}}$
$\newcommand{\sysParamURcSubOne}{-0.28}$
$\newcommand{\sysParamURcSubOneUnc}{_{-0.20}^{+0.33}}$
$\newcommand{\sysParamUZsSubZero}{0.42}$
$\newcommand{\sysParamUZsSubZeroUnc}{_{-0.28}^{+0.34}}$
$\newcommand{\sysParamUZsSubOne}{0.01}$
$\newcommand{\sysParamUZsSubOneUnc}{_{-0.27}^{+0.32}}$
$\newcommand{\sysParamMeanFluxZero}{(-1.3}$
$\newcommand{\sysParamMeanFluxZeroUnc}{\pm 1.2) \times 10^{-5}}$
$\newcommand{\sysParamMeanFluxOne}{(1.5}$
$\newcommand{\sysParamMeanFluxOneUnc}{\pm 1.2) \times 10^{-5}}$
$\newcommand{\sysParamLcJitterSubZero}{(9.2}$
$\newcommand{\sysParamLcJitterSubZeroUnc}{_{-4.2}^{+3.1}) \times 10^{-5}}$
$\newcommand{\sysParamLcJitterSubOne}{(2.08}$
$\newcommand{\sysParamLcJitterSubOneUnc}{_{-1.15}^{+0.86}) \times 10^{-4}}$
$\newcommand{\sysParamLcJitterSubTwo}{(1.26}$
$\newcommand{\sysParamLcJitterSubTwoUnc}{_{-0.88}^{+1.24}) \times 10^{-4}}$
$\newcommand{\sysParamLcJitterSubThree}{0.00124}$
$\newcommand{\sysParamLcJitterSubThreeUnc}{\pm 0.00014}$
$\newcommand{\sysParamRvGammaSubZero}{6648.3}$
$\newcommand{\sysParamRvGammaSubZeroUnc}{_{-13.1}^{+9.5}}$
$\newcommand{\sysParamRvGammaSubOne}{8114}$
$\newcommand{\sysParamRvGammaSubOneUnc}{_{-14}^{+11}}$
$\newcommand{\sysParamRvGammaSubTwo}{8118.5}$
$\newcommand{\sysParamRvGammaSubTwoUnc}{_{-12.7}^{+8.9}}$
$\newcommand{\sysParamRvJitterSubZero}{3.2}$
$\newcommand{\sysParamRvJitterSubZeroUnc}{_{-2.2}^{+3.1}}$
$\newcommand{\sysParamRvJitterSubOne}{14.1}$
$\newcommand{\sysParamRvJitterSubOneUnc}{_{-3.4}^{+4.4}}$
$\newcommand{\sysParamRvJitterSubTwo}{3.16}$
$\newcommand{\sysParamRvJitterSubTwoUnc}{_{-0.78}^{+0.83}}$
$\newcommand{\sysParamGpSigma}{22.0}$
$\newcommand{\sysParamGpSigmaUnc}{_{-7.9}^{+14.6}}$
$\newcommand{\sysParamGpRho}{118}$
$\newcommand{\sysParamGpRhoUnc}{_{-40}^{+54}}$
$\newcommand{\sysParamGpTau}{33}$
$\newcommand{\sysParamGpTauUnc}{_{-28}^{+94}}$
$\newcommand{\sysParamRPlanetJupiterSubZero}{0.711}$
$\newcommand{\sysParamRPlanetJupiterSubZeroUnc}{\pm 0.011}$
$\newcommand{\sysParamRPlanetJupiterSubOne}{0.236}$
$\newcommand{\sysParamRPlanetJupiterSubOneUnc}{_{-0.011}^{+0.010}}$
$\newcommand{\sysParamMPlanetJupiterSubZero}{0.238}$
$\newcommand{\sysParamMPlanetJupiterSubZeroUnc}{\pm 0.012}$
$\newcommand{\sysParamMPlanetJupiterSubOne}{0.0324}$
$\newcommand{\sysParamMPlanetJupiterSubOneUnc}{\pm 0.0070}$
$\newcommand{\sysParamTempEqSubZero}{1029}$
$\newcommand{\sysParamTempEqSubZeroUnc}{_{-111}^{+84}}$
$\newcommand{\sysParamTempEqSubOne}{1475}$
$\newcommand{\sysParamTempEqSubOneUnc}{_{-158}^{+120}}$
$\newcommand{\sysParamOmegaFoldSubZero}{200}$
$\newcommand{\sysParamOmegaFoldSubZeroUnc}{_{-34}^{+27}}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.05}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\plusminus$}[3]{#1^{+#2}_{-#3}}$
$\newcommand{$\circa$}{$\ensuremath${{\sim}\mspace{2mu}}}$
$\newcommand{$\ut$}{$\textsc${ut}}$
$\newcommand{$\tablenotemark$}[1]{\textsuperscript{\textit{#1}}}$
$\newcommand{$\bjdtdb$}{$\ensuremath${\mathrm{BJD}_\text{TDB}}}$
$\newcommand{$\feh$}{$\ensuremath${[\text{Fe}/\text{H}]}}$
$\newcommand{$\logg$}{$\ensuremath${\log g}}$
$\newcommand{$\tempeff$}{$\ensuremath${T_{\text{eff}}}}$
$\newcommand{$\tempeq$}{$\ensuremath${T_{\text{eq}}}}$
$\newcommand{$\vsini$}{$\ensuremath${v \sin i}}$
$\newcommand{$\normaldist$}{\mathcal{N}}$
$\newcommand{$\uniformdist$}{\mathcal{U}}$
$\newcommand{$\gaiaG$}{$\ensuremath${G}}$
$\newcommand{$\gaiaBP$}{$\ensuremath${G_\mathrm{BP}}}$
$\newcommand{$\gaiaRP$}{$\ensuremath${G_\mathrm{RP}}}$
$\newcommand{$\filterB$}{$\ensuremath${B}}$
$\newcommand{$\filterIc$}{$\ensuremath${I_\mathrm{c}}}$
$\newcommand{$\filterJ$}{$\ensuremath${J}}$
$\newcommand{$\filterH$}{$\ensuremath${H}}$
$\newcommand{$\filterK$}{$\ensuremath${K_\mathrm{s}}}$
$\newcommand{$\filterR$}{$\ensuremath${R}}$
$\newcommand{$\filterR$c}{$\ensuremath${R_\mathrm{c}}}$
$\newcommand{$\filterV$}{$\ensuremath${V}}$
$\newcommand{$\filterWone$}{$\ensuremath${W1}}$
$\newcommand{$\filterWtwo$}{$\ensuremath${W2}}$
$\newcommand{$\filterWthree$}{$\ensuremath${W3}}$
$\newcommand{$\filterWfour$}{$\ensuremath${W4}}$
$\newcommand{$\filtergp$}{$\ensuremath${g'}}$
$\newcommand{$\filterip$}{$\ensuremath${i'}}$
$\newcommand{$\filterrp$}{$\ensuremath${r'}}$
$\newcommand{$\filterzs$}{$\ensuremath${z_\mathrm{s}}}$
$\newcommand{$\ares$}{$\textsc${ares}}$
$\newcommand{$\arviz$}{$\textsc${arviz}}$
$\newcommand{$\astroimagej$}{$\textsc${AstroImageJ}}$
$\newcommand{$\batman$}{$\textsc${batman}}$
$\newcommand{$\celeritetwo$}{$\textsc${celerite2}}$
$\newcommand{$\emcee$}{$\textsc${emcee}}$
$\newcommand{$\exofast$}{$\textsc${exofast}}$
$\newcommand{$\exoplanetpy$}{$\textsc${exoplanet}}$
$\newcommand{$\isochrones$}{$\textsc${isochrones}}$
$\newcommand{$\mist$}{MIST}$
$\newcommand{$\molusc$}{$\textsc${molusc}}$
$\newcommand{$\moog$}{$\textsc${moog}}$
$\newcommand{$\pymc$}{$\textsc${PyMC}}$
$\newcommand{$\qlp$}{$\textsc${qlp}}$
$\newcommand{$\radvel$}{$\textsc${radvel}}$
$\newcommand{$\sklearn$}{$\textsc${Scikit-learn}}$
$\newcommand{$\triceratops$}{$\textsc${triceratops}}$
$\newcommand{$\vartools$}{$\textsc${Vartools}}$
$\newcommand{$\planet$}[2]{#1\thinspace #2}$
$\newcommand{$\toitwothousand$}{TOI-2000}$
$\newcommand{$\toitwothousand$tic}{371188886}$
$\newcommand{$\planet$inner}{$\planet${$\toitwothousand$}{b}}$
$\newcommand{$\planet$outer}{$\planet${$\toitwothousand$}{c}}$
$\newcommand{$\waspfortyseven$}{WASP-47}$
$\newcommand{$\waspfortyseven$hj}{$\planet${$\waspfortyseven$}{b}}$
$\newcommand{$\waspfortyseven$inner}{$\planet${$\waspfortyseven$}{e}}$
$\newcommand{$\waspfortyseven$neptune}{$\planet${$\waspfortyseven$}{d}}$
$\newcommand{$\keplerseventhirty$}{Kepler-730}$
$\newcommand{$\keplerseventhirty$hj}{$\planet${Kepler-730}{b}}$
$\newcommand{$\keplerseventhirty$inner}{$\planet${Kepler-730}{c}}$
$\newcommand{$\toieleventhirty$}{TOI-1130}$
$\newcommand{$\toieleventhirty$inner}{$\planet${$\toieleventhirty$}{b}}$
$\newcommand{$\toieleventhirty$hj}{$\planet${$\toieleventhirty$}{c}}$
$\newcommand{$\keplerthirty$}{Kepler-30}$
$\newcommand{$\keplerthirty$b}{$\planet${$\keplerthirty$}{b}}$
$\newcommand{$\keplereightynine$}{KOI-94}$
$\newcommand{$\keplereightynine$b}{$\planet${$\keplereightynine$}{b}}$
$\newcommand{$\wasponethirtytwo$}{WASP-132}$
$\newcommand{$\wasponethirtytwo$hj}{$\planet${WASP-132}{b}}$
$\newcommand{$\wasponethirtytwo$inner}{$\planet${WASP-132}{c}}$
$\newcommand{$\astep$}{ASTEP}$
$\newcommand{$\chiron$}{CHI\-RON}$
$\newcommand{$\edrthree$}{EDR3}$
$\newcommand{$\feros$}{FEROS}$
$\newcommand{$\gaia$}{\emph{Gaia}}$
$\newcommand{$\geminisouth$}{Gemini South}$
$\newcommand{$\harps$}{HARPS}$
$\newcommand{$\jwst$}{\emph{JWST}}$
$\newcommand{$\kepler$}{\emph{Kepler}}$
$\newcommand{$\ktwo$}{\emph{K2}}$
$\newcommand{$\lcogt$}{LCOGT}$
$\newcommand{$\pest$}{PEST}$
$\newcommand{$\soar$}{SOAR}$
$\newcommand{$\tess$}{\emph{TESS}}$
$\newcommand{$\twomass$}{2MASS}$
$\newcommand{$\wise$}{\emph{WISE}}$
$\newcommand{$\sysParamPeriodSubZero$}{9.1270553}$
$\newcommand{$\sysParamPeriodSubZero$Unc}{\pm 0.0000073}$
$\newcommand{$\sysParamPeriodSubOne$}{3.098331}$
$\newcommand{$\sysParamPeriodSubOne$Unc}{_{-0.000019}^{+0.000021}}$
$\newcommand{$\sysParamTzeroSubZero$}{2459110.06589}$
$\newcommand{$\sysParamTzeroSubZero$Unc}{\pm 0.00027}$
$\newcommand{$\sysParamTzeroSubOne$}{2458901.7192}$
$\newcommand{$\sysParamTzeroSubOne$Unc}{_{-0.0020}^{+0.0021}}$
$\newcommand{$\sysParamRpSubZero$}{0.06564}$
$\newcommand{$\sysParamRpSubZero$Unc}{\pm 0.00064}$
$\newcommand{$\sysParamRpSubOne$}{0.02178}$
$\newcommand{$\sysParamRpSubOne$Unc}{_{-0.00096}^{+0.00089}}$
$\newcommand{$\sysParamBSubZero$}{0.618}$
$\newcommand{$\sysParamBSubZero$Unc}{_{-0.042}^{+0.037}}$
$\newcommand{$\sysParamBSubOne$}{0.763}$
$\newcommand{$\sysParamBSubOne$Unc}{_{-0.072}^{+0.033}}$
$\newcommand{$\sysParamTdurSubZero$}{3.90}$
$\newcommand{$\sysParamTdurSubZero$Unc}{\pm 0.40}$
$\newcommand{$\sysParamTdurSubOne$}{1.957}$
$\newcommand{$\sysParamTdurSubOne$Unc}{_{-0.099}^{+0.208}}$
$\newcommand{$\sysParamSqrtEccVecZeroSubZero$}{-0.213}$
$\newcommand{$\sysParamSqrtEccVecZeroSubZero$Unc}{_{-0.046}^{+0.060}}$
$\newcommand{$\sysParamSqrtEccVecZeroSubOne$}{-0.08}$
$\newcommand{$\sysParamSqrtEccVecZeroSubOne$Unc}{_{-0.11}^{+0.13}}$
$\newcommand{$\sysParamEccSubZero$}{0.064}$
$\newcommand{$\sysParamEccSubZero$Unc}{_{-0.022}^{+0.023}}$
$\newcommand{$\sysParamOmegaSubZero$}{-2.43}$
$\newcommand{$\sysParamOmegaSubZero$Unc}{_{-0.46}^{+5.19}}$
$\newcommand{$\sysParamRPlanetEarthSubZero$}{7.97}$
$\newcommand{$\sysParamRPlanetEarthSubZero$Unc}{\pm 0.12}$
$\newcommand{$\sysParamRPlanetEarthSubOne$}{2.64}$
$\newcommand{$\sysParamRPlanetEarthSubOne$Unc}{_{-0.12}^{+0.11}}$
$\newcommand{$\sysParamMPlanetEarthSubZero$}{75.7}$
$\newcommand{$\sysParamMPlanetEarthSubZero$Unc}{\pm 3.8}$
$\newcommand{$\sysParamMPlanetEarthSubOne$}{10.3}$
$\newcommand{$\sysParamMPlanetEarthSubOne$Unc}{\pm 2.2}$
$\newcommand{$\sysParamMPlanetSubZero$}{(2.27}$
$\newcommand{$\sysParamMPlanetSubZero$Unc}{_{-0.12}^{+0.11}) \times 10^{-4}}$
$\newcommand{$\sysParamMPlanetSubOne$}{(3.09}$
$\newcommand{$\sysParamMPlanetSubOne$Unc}{_{-0.66}^{+0.67}) \times 10^{-5}}$
$\newcommand{$\sysParamRhoPlanetSubZero$}{0.822}$
$\newcommand{$\sysParamRhoPlanetSubZero$Unc}{_{-0.056}^{+0.058}}$
$\newcommand{$\sysParamRhoPlanetSubOne$}{3.06}$
$\newcommand{$\sysParamRhoPlanetSubOne$Unc}{_{-0.72}^{+0.83}}$
$\newcommand{$\sysParamKSubZero$}{23.8}$
$\newcommand{$\sysParamKSubZero$Unc}{\pm 1.1}$
$\newcommand{$\sysParamKSubOne$}{4.63}$
$\newcommand{$\sysParamKSubOne$Unc}{_{-0.99}^{+1.00}}$
$\newcommand{$\sysParamASubZero$}{0.0875}$
$\newcommand{$\sysParamASubZero$Unc}{_{-0.0014}^{+0.0016}}$
$\newcommand{$\sysParamASubOne$}{0.04260}$
$\newcommand{$\sysParamASubOne$Unc}{_{-0.00067}^{+0.00076}}$
$\newcommand{$\sysParamInclSubZero$}{87.94}$
$\newcommand{$\sysParamInclSubZero$Unc}{_{-0.11}^{+0.13}}$
$\newcommand{$\sysParamInclSubOne$}{84.73}$
$\newcommand{$\sysParamInclSubOne$Unc}{_{-0.33}^{+0.49}}$
$\newcommand{$\sysParamAorSubZero$}{16.92}$
$\newcommand{$\sysParamAorSubZero$Unc}{_{-0.33}^{+0.37}}$
$\newcommand{$\sysParamAorSubOne$}{8.23}$
$\newcommand{$\sysParamAorSubOne$Unc}{_{-0.16}^{+0.18}}$
$\newcommand{$\sysParamMStarZero$}{1.074}$
$\newcommand{$\sysParamMStarZero$Unc}{_{-0.050}^{+0.059}}$
$\newcommand{$\sysParamFehZero$}{0.414}$
$\newcommand{$\sysParamFehZero$Unc}{_{-0.041}^{+0.040}}$
$\newcommand{$\sysParamEep$}{380}$
$\newcommand{$\sysParamEep$Unc}{_{-36}^{+25}}$
$\newcommand{$\sysParamMStar$}{1.074}$
$\newcommand{$\sysParamMStar$Unc}{_{-0.050}^{+0.059}}$
$\newcommand{$\sysParamRStar$}{1.113}$
$\newcommand{$\sysParamRStar$Unc}{\pm 0.011}$
$\newcommand{$\sysParamRhoStar$}{1.099}$
$\newcommand{$\sysParamRhoStar$Unc}{_{-0.063}^{+0.074}}$
$\newcommand{$\sysParamLoggStar$}{4.376}$
$\newcommand{$\sysParamLoggStar$Unc}{_{-0.023}^{+0.026}}$
$\newcommand{$\sysParamTeff$}{5607}$
$\newcommand{$\sysParamTeff$Unc}{_{-82}^{+86}}$
$\newcommand{$\sysParamFeh$}{0.438}$
$\newcommand{$\sysParamFeh$Unc}{_{-0.042}^{+0.041}}$
$\newcommand{$\sysParamAge$}{5.1}$
$\newcommand{$\sysParamAge$Unc}{\pm 2.7}$
$\newcommand{$\sysParamParallax$}{5.760}$
$\newcommand{$\sysParamParallax$Unc}{\pm 0.010}$
$\newcommand{$\sysParamAv$}{0.156}$
$\newcommand{$\sysParamAv$Unc}{_{-0.072}^{+0.074}}$
$\newcommand{$\sysParamMagBol$}{4.634}$
$\newcommand{$\sysParamMagBol$Unc}{_{-0.054}^{+0.052}}$
$\newcommand{$\sysParamLStar$}{1.103}$
$\newcommand{$\sysParamLStar$Unc}{_{-0.051}^{+0.056}}$
$\newcommand{$\sysParamDistance$}{173.60}$
$\newcommand{$\sysParamDistance$Unc}{\pm 0.31}$
$\newcommand{$\sysParamSedUncScale$}{1.30}$
$\newcommand{$\sysParamSedUncScale$Unc}{_{-0.22}^{+0.41}}$
$\newcommand{$\sysParamIrradiationSubZero$}{(1.961}$
$\newcommand{$\sysParamIrradiationSubZero$Unc}{_{-0.094}^{+0.101}) \times 10^{8}}$
$\newcommand{$\sysParamIrradiationSubOne$}{(8.26}$
$\newcommand{$\sysParamIrradiationSubOne$Unc}{_{-0.40}^{+0.43}) \times 10^{8}}$
$\newcommand{$\sysParamUTessSubZero$}{0.124}$
$\newcommand{$\sysParamUTessSubZero$Unc}{_{-0.090}^{+0.163}}$
$\newcommand{$\sysParamUTessSubOne$}{0.34}$
$\newcommand{$\sysParamUTessSubOne$Unc}{_{-0.23}^{+0.16}}$
$\newcommand{$\sysParamURcSubZero$}{0.88}$
$\newcommand{$\sysParamURcSubZero$Unc}{_{-0.28}^{+0.21}}$
$\newcommand{$\sysParamURcSubOne$}{-0.28}$
$\newcommand{$\sysParamURcSubOne$Unc}{_{-0.20}^{+0.33}}$
$\newcommand{$\sysParamUZsSubZero$}{0.42}$
$\newcommand{$\sysParamUZsSubZero$Unc}{_{-0.28}^{+0.34}}$
$\newcommand{$\sysParamUZsSubOne$}{0.01}$
$\newcommand{$\sysParamUZsSubOne$Unc}{_{-0.27}^{+0.32}}$
$\newcommand{$\sysParamMeanFluxZero$}{(-1.3}$
$\newcommand{$\sysParamMeanFluxZero$Unc}{\pm 1.2) \times 10^{-5}}$
$\newcommand{$\sysParamMeanFluxOne$}{(1.5}$
$\newcommand{$\sysParamMeanFluxOne$Unc}{\pm 1.2) \times 10^{-5}}$
$\newcommand{$\sysParamLcJitterSubZero$}{(9.2}$
$\newcommand{$\sysParamLcJitterSubZero$Unc}{_{-4.2}^{+3.1}) \times 10^{-5}}$
$\newcommand{$\sysParamLcJitterSubOne$}{(2.08}$
$\newcommand{$\sysParamLcJitterSubOne$Unc}{_{-1.15}^{+0.86}) \times 10^{-4}}$
$\newcommand{$\sysParamLcJitterSubTwo$}{(1.26}$
$\newcommand{$\sysParamLcJitterSubTwo$Unc}{_{-0.88}^{+1.24}) \times 10^{-4}}$
$\newcommand{$\sysParamLcJitterSubThree$}{0.00124}$
$\newcommand{$\sysParamLcJitterSubThree$Unc}{\pm 0.00014}$
$\newcommand{$\sysParamRvGammaSubZero$}{6648.3}$
$\newcommand{$\sysParamRvGammaSubZero$Unc}{_{-13.1}^{+9.5}}$
$\newcommand{$\sysParamRvGammaSubOne$}{8114}$
$\newcommand{$\sysParamRvGammaSubOne$Unc}{_{-14}^{+11}}$
$\newcommand{$\sysParamRvGammaSubTwo$}{8118.5}$
$\newcommand{$\sysParamRvGammaSubTwo$Unc}{_{-12.7}^{+8.9}}$
$\newcommand{$\sysParamRvJitterSubZero$}{3.2}$
$\newcommand{$\sysParamRvJitterSubZero$Unc}{_{-2.2}^{+3.1}}$
$\newcommand{$\sysParamRvJitterSubOne$}{14.1}$
$\newcommand{$\sysParamRvJitterSubOne$Unc}{_{-3.4}^{+4.4}}$
$\newcommand{$\sysParamRvJitterSubTwo$}{3.16}$
$\newcommand{$\sysParamRvJitterSubTwo$Unc}{_{-0.78}^{+0.83}}$
$\newcommand{$\sysParamGpSigma$}{22.0}$
$\newcommand{$\sysParamGpSigma$Unc}{_{-7.9}^{+14.6}}$
$\newcommand{$\sysParamGpRho$}{118}$
$\newcommand{$\sysParamGpRho$Unc}{_{-40}^{+54}}$
$\newcommand{$\sysParamGpTau$}{33}$
$\newcommand{$\sysParamGpTau$Unc}{_{-28}^{+94}}$
$\newcommand{$\sysParamRPlanetJupiterSubZero$}{0.711}$
$\newcommand{$\sysParamRPlanetJupiterSubZero$Unc}{\pm 0.011}$
$\newcommand{$\sysParamRPlanetJupiterSubOne$}{0.236}$
$\newcommand{$\sysParamRPlanetJupiterSubOne$Unc}{_{-0.011}^{+0.010}}$
$\newcommand{$\sysParamMPlanetJupiterSubZero$}{0.238}$
$\newcommand{$\sysParamMPlanetJupiterSubZero$Unc}{\pm 0.012}$
$\newcommand{$\sysParamMPlanetJupiterSubOne$}{0.0324}$
$\newcommand{$\sysParamMPlanetJupiterSubOne$Unc}{\pm 0.0070}$
$\newcommand{$\sysParamTempEqSubZero$}{1029}$
$\newcommand{$\sysParamTempEqSubZero$Unc}{_{-111}^{+84}}$
$\newcommand{$\sysParamTempEqSubOne$}{1475}$
$\newcommand{$\sysParamTempEqSubOne$Unc}{_{-158}^{+120}}$
$\newcommand{$\sysParamOmegaFoldSubZero$}{200}$
$\newcommand{$\sysParamOmegaFoldSubZero$Unc}{_{-34}^{+27}}$
$\newcommand{$\arraystretch$}{1.25}$
$\newcommand{$\arraystretch$}{1.25}$
$\newcommand{$\arraystretch$}{1.25}$
$\newcommand{$\arraystretch$}{1.25}$
$\newcommand{$\arraystretch$}{1.05}$</div>



<div id="title">

#  spots a mini-neptune interior to a hot saturn in the TOI-2000 system

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.14396-b31b1b.svg)](https://arxiv.org/abs/2209.14396)<mark>Appeared on: 2022-09-30</mark> - _20 pages, 7 figures, 13 tables; submitted to MNRAS; CSV tables available as ancillary files; posterior samples available from Zenodo at this https URL_

</div>
<div id="authors">

<mark><mark>Lizhou Sha</mark></mark>, et al. -- incl., <mark><mark>Melissa J. Hobson</mark></mark>, <mark><mark>Carl Ziegler</mark></mark>, <mark><mark>Pascal Torres-Miranda</mark></mark>, <mark><mark>Trifon Trifonov</mark></mark>, <mark><mark>Colin Littlefield</mark></mark>, <mark><mark>Jan Eberhardt</mark></mark>, <mark><mark>Richard P. Schwarz</mark></mark>, <mark><mark>Thomas Henning</mark></mark>, <mark><mark>Jack J. Lissauer</mark></mark>

</div>
<div id="abstract">

**Abstract:** Hot jupiters ($P < 10 \text{d}, M > 60  \mathrm{M}_\oplus$)are almost always found alone around their stars,but four out of hundreds known have inner companion planets.These rare companions allow us to constrain the hot jupiter's formation historyby ruling out high-eccentricity tidal migration.Less is known about inner companions to hot Saturn-mass planets.We report here the discovery of the TOI-2000 system, which features a hot Saturn-mass planet with a smaller innercompanion.The mini-neptune TOI-2000\thinspaceb($\sysParamRPlanetEarthSubOne$$\sysParamRPlanetEarthSubOne$Unc$  \mathrm{R}_\oplus$,$\sysParamMPlanetEarthSubOne$$\sysParamMPlanetEarthSubOne$Unc$  \mathrm{M}_\oplus$)is in a3.10-day orbit,and the hot saturn TOI-2000\thinspacec($\sysParamRPlanetEarthSubZero$$\sysParamRPlanetEarthSubZero$Unc$  \mathrm{R}_\oplus$,$\sysParamMPlanetEarthSubZero$$\sysParamMPlanetEarthSubZero$Unc$  \mathrm{M}_\oplus$)is in a9.13-day orbit.Both planets transit their host star TOI-2000(TIC\thinspace371188886,$V = 10.98$,\textit{TESS}magnitude$= 10.36$),a metal-rich ($[\text{Fe}/\text{H}] =$\sysParamFeh$\sysParamFeh$Unc)G dwarf174\thinspacepc away.\emph{TESS}observed the two planets in sectors 9–11 and 36–38,and we followed up with ground-based photometry, spectroscopy, and speckle imaging.Radial velocities from HARPS allowed us to confirm both planets by direct mass measurement.In addition, we demonstrate constrainingplanetary and stellar parameterswith MIST stellar evolutionary tracksthrough Hamiltonian Monte Carlo under the PyMC framework,achieving higher sampling efficiency and shorter run timecompared to traditional Markov chain Monte Carlo.Having the brightest host star in the$V$band among similar systems,TOI-2000\thinspaceb and c are superb candidates for atmospheric characterization by the\emph{JWST},which can potentially distinguish whether they formed together orTOI-2000\thinspacec swept along material during migration to form TOI-2000\thinspaceb.

</div>

<div id="div_fig1">

<img src="tmp_2209.14396/./figures/family_portrait.png" alt="Fig4" width="100%"/>

**Figure 4. -** Transiting planetary systems hosting small planets orbiting interior to giant planets ($P < 100 \text{d}$).
    Non-transiting planets in these systems are omitted.
    The leftmost circle in each row represent the host star,
    with the mark's linear size proportional to the stellar radius
    and fill colour indicating the effective temperature
    \citep[from TIC\thinspace 8.1;][]{tess_tic8}.
    The solid circles represent the transiting planets in the system,
    with their linear sizes proportional to planetary radii
    and fill colours indicating whether the planet is a
    giant planet (orange, $P < 10 \text{d}$; blue, $P \geq 10 \text{d}$)
    or a small planet (grey, $R < \SI{6}{\radius\earth}$).
    The relative mark sizes among either the stars or the planets are to scale,
    but not between a star and its planets.
    The slightly translucent marks are planet candidates,
    and systems that contain them are marked with an asterisk.
    The systems are sorted in ascending order of the period of the largest planet from top to bottom.
    \autoref{tab:giant_multi} contains the numbers and references underlying this figure.
     (*fig:family*)

</div>
<div id="div_fig2">

<img src="tmp_2209.14396/./figures/toi_2000_lc.png" alt="Fig5" width="100%"/>

**Figure 5. -** Light curves of $\toitwothousand$ .
    Top two panels: Detrended light curves from $\tess$  years 1 and 3.
    Year 1 points are from the 30-min full-frame images,
    whilst year 3 points are binned to 2 min from the 20-s time series,
    which results in higher scatter per cadence but lower scatter at 30 min.
    The upright triangles indicate transits of planet b (blue) and planet c (orange).
    Bottom panels:
    Phase-folded light curves zoomed in on the transits of planets b (third row)
    and c (last row).
    The purple line is the transit model using limb darkening parameters in the $\tess$  band.
    The faint grey marks are $\tess$  observations from both years,
    with year 3 observations binned to 30 min to be consistent with those from year 1.
    The orange marks are means within 12-min bins,
    with the error bars representing the standard error of the mean.
    The bottom right panel includes planet c full-transit light curves from ground observations,
    and the model shown uses a different set of limb darkening parameters in $\filterR$c .
    Because the ground observations have different uncertainties,
    the binned mean is weighted by inverse variance
    and the error bar is the standard error of the weighted mean.
     (*fig:lc*)

</div>
<div id="div_fig3">

<img src="tmp_2209.14396/./figures/toi_2000_mass_radius.png" alt="Fig7" width="100%"/>

**Figure 7. -** Mass--radius relationship of small exoplanets ($< \SI{4}{\radius\earth}$).
    Only planets with mass determination better than 33\% have been included.
    The values used are the \enquote{default parameter set} of the Planetary Systems Table
    from the NASA Exoplanet Archive \citep[][accessed on 2022 August 19]{nasa_exoplanet_archive,nasa_exoplanet_archive_ps}.
    The mini-Neptune $\planet$inner ,
    together with four small planets
    $\planet${$\waspfortyseven$}{d} and e \citep{2022AJ....163..197B},
    $\toieleventhirty$inner(Korth et al., submitted),
    $\planet${Kepler-30}{b}\citep{2012Natur.487..449S},
    and $\planet${KOI-94}{b}\citep[also known as Kepler-89;][]{2013ApJ...768...14W},
    which are also found in systems with giant planets,
    are highlighted, with error bars representing quoted uncertainties.
    The single letters denote the Solar System planets Venus, Earth, Uranus, and Neptune.
    Three theoretical mass--radius curves by
    \citet[available online at \url{https://lweb.cfa.harvard.edu/ lzeng/planetmodels.html}]{2019PNAS..116.9723Z}
    are plotted for reference:
    solid brown for an Earth-like rocky core (32.5\% Fe and 67.5\%$\text{MgSiO}_3$),
    dotted-dash blue for an 100\% water world at \SI{1000}{\kelvin},
    and dotted teal for a planet with 1\% hydrogen envelope and 99\% Earth-like rocky core at \SI{1000}{\kelvin}.
     (*fig:massradius*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\mbh}{{\mbox{M_\mathrm{BH}}}}$
$\newcommand{\hii}{\mbox{H{\sevenrm II}}}$
$\newcommand{\ha}{{\mbox{H\alpha}}}$
$\newcommand{\hb}{{\mbox{H\beta}}}$
$\newcommand{\brg}{{\mbox{Br\gamma}}}$
$\newcommand{\paa}{{\mbox{Pa\alpha}}}$
$\newcommand{\civ}{[C{\sevenrm IV}]}$
$\newcommand{\heiia}{He{\sevenrm II} \lambda4686}$
$\newcommand{\heii}{He{\sevenrm II}}$
$\newcommand{\hei}{He{\sevenrm I}}$
$\newcommand{\feii}{Fe{\sevenrm II}}$
$\newcommand{\OIIIa}{[O{\sevenrm III}] \lambda4959}$
$\newcommand{\OIIIb}{[O{\sevenrm III}] \lambda5007}$
$\newcommand{\OIIIc}{[O{\sevenrm III}] \lambda\lambda4959,5007}$
$\newcommand{\OIII}{[O{\sevenrm III}]}$
$\newcommand{\NI}{[N{\sevenrm I}]}$
$\newcommand{\uas}{\mbox{\muas}}$
$\newcommand{\lopt}{\mbox{\lambda L_\lambda (5100 \AA)}}$
$\newcommand{\Rd}{\mbox{R_\mathrm{d}}}$
$\newcommand{\Rb}{\mbox{R_\mathrm{BLR}}}$
$\newcommand{\rEdd}{\mbox{\lambda_\mathrm{Edd}}}$
$\newcommand{\rl}{\mbox{R--L}}$
$\newcommand{\nodata}{{...}}$
$\newcommand{\mcl}{\multicolumn}$
$\newcommand{\um}{{\mbox{\mum}}}$
$\newcommand{\kms}{{\mbox{\mathrm{km s^{-1}}}}}$
$\newcommand{\ergs}{{\mbox{\mathrm{erg s^{-1}}}}}$
$\newcommand{\msun}{{\mbox{M_\odot}}}$
$\newcommand{\uhc}{{\mbox{\mathrm{km s^{-1}~Mpc^{-1}}}}}$
$\newcommand{\degree}{{\mbox{^\circ}}}$
$\newcommand{\hi}{H{\sevenrm I}}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.2}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\mbh$}{{\mbox{M_\mathrm{BH}}}}$
$\newcommand{$\hii$}{\mbox{H{\sevenrm II}}}$
$\newcommand{$\ha$}{{\mbox{H\alpha}}}$
$\newcommand{$\hb$}{{\mbox{H\beta}}}$
$\newcommand{$\brg$}{{\mbox{Br\gamma}}}$
$\newcommand{$\paa$}{{\mbox{Pa\alpha}}}$
$\newcommand{$\civ$}{[C{\sevenrm IV}]}$
$\newcommand{$\heiia$}{He{\sevenrm II} \lambda4686}$
$\newcommand{$\heii$}{He{\sevenrm II}}$
$\newcommand{$\hei$}{He{\sevenrm I}}$
$\newcommand{$\feii$}{Fe{\sevenrm II}}$
$\newcommand{$\OIIIa$}{[O{\sevenrm III}] \lambda4959}$
$\newcommand{$\OIIIb$}{[O{\sevenrm III}] \lambda5007}$
$\newcommand{$\OIIIc$}{[O{\sevenrm III}] \lambda\lambda4959,5007}$
$\newcommand{$\OIII$}{[O{\sevenrm III}]}$
$\newcommand{$\NI$}{[N{\sevenrm I}]}$
$\newcommand{$\uas$}{\mbox{\muas}}$
$\newcommand{$\lopt$}{\mbox{\lambda L_\lambda (5100 \AA)}}$
$\newcommand{$\Rd$}{\mbox{R_\mathrm{d}}}$
$\newcommand{$\Rb$}{\mbox{R_\mathrm{BLR}}}$
$\newcommand{$\rEdd$}{\mbox{\lambda_\mathrm{Edd}}}$
$\newcommand{$\rl$}{\mbox{R--L}}$
$\newcommand{$\nodata$}{{...}}$
$\newcommand{$\mcl$}{\multicolumn}$
$\newcommand{$\um$}{{\mbox{\mum}}}$
$\newcommand{$\kms$}{{\mbox{\mathrm{km s^{-1}}}}}$
$\newcommand{$\ergs$}{{\mbox{\mathrm{erg s^{-1}}}}}$
$\newcommand{$\msun$}{{\mbox{M_\odot}}}$
$\newcommand{$\uhc$}{{\mbox{\mathrm{km s^{-1}~Mpc^{-1}}}}}$
$\newcommand{$\degree$}{{\mbox{^\circ}}}$
$\newcommand{$\hi$}{H{\sevenrm I}}$
$\newcommand{$\arraystretch$}{1.2}$
$\newcommand{$\arraystretch$}{1.5}$
$\newcommand{$\arraystretch$}{1.2}$</div>



<div id="title">

# Towards measuring supermassive black hole masses with interferometric observations of the dust continuum

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.14410-b31b1b.svg)](https://arxiv.org/abs/2209.14410)<mark>Appeared on: 2022-09-30</mark> - _Accepted for publication in A&A; 11 pages, 5 figures, 3 tables_

</div>
<div id="authors">

GRAVITY Collaboration, et al. -- incl., <mark><mark>R. Davies</mark></mark>, <mark><mark>N. M. Förster Schreiber</mark></mark>

</div>
<div id="abstract">

**Abstract:** This work focuses on active galactic nuclei (AGNs), and the relationbetween the sizes of the hot dust continuum and the broad-line region (BLR).We find that the continuum size measured using optical/near-infraredinterferometry (OI) is roughly twice that measured by reverberation mapping (RM).Both OI and RM continuum sizes show a tight relation with the$\hb$BLRsize with only an intrinsic scatter of 0.25 dex.  The massesof supermassive black holes (BHs) can hence be simply derived from a dust sizein combination with a broad line width and virial factor. Since the primaryuncertainty of these BH masses comes from the virial factor, the accuracy ofthe continuum-based BH masses is close to those based on the RM measurement ofthe broad emission line.Moreover, the necessary continuum measurements can be obtained on a much shortertimescale than those required monitoring for RM, and are also more time efficientthan those needed to resolve the BLR with OI.  The primary goal of thiswork is to demonstrate measuring the BH mass based on the dust continuum sizewith our first calibration of the$\Rb$--$\Rd$relation.  The current limitationand caveats are discussed in detail.  Future GRAVITY observations are expectedto improve the continuum-based method and have the potential to measure BHmasses for a large sample of AGNs in the low-redshift Universe.

</div>

<div id="div_fig1">

<img src="tmp_2209.14410/./fig2.png" alt="Fig3" width="100%"/>

**Figure 3. -** Tight relations between the radii of the BLR and the dust continuum.
The blue circles are AGNs measured by RM, and the red diamonds are the targets
measured by OI.  The gray dashed lines connect the same sources with
both RM and OI measured $\Rd$.  The best-fit $\Rb$--$\Rd$ relations of RM and OI
datasets (formula in the lower right corner of each panel) are shown as the blue
and red lines, respectively.  The black circles enclose the known changing-look
AGNs.  We only exclude Mrk 590, denoted as the empty blue circle, in our
analysis.  The best-fit results with the slope free (a) or fixed to unity (b)
are almost the same, so we mainly adopt the results of (b) in the discussion.
The linear scale is indicated on the top and right axes. (*fig:sc*)

</div>
<div id="div_fig2">

<img src="tmp_2209.14410/./fig1.png" alt="Fig2" width="100%"/>

**Figure 2. -** Size--luminosity relations of the dust continuum measured by
(a) RM (blue circles) and (b) OI (red diamonds) are systematically above
that of the BLR (black circles; same in both panels).
The open-colored circles indicate sources without BLR measurements.
The dashed lines in both panels are based on RM-measured $\rl$ relations of the dust continuum \citep[blue;][]{GC2020cont} and
the BLR \citep[black;][]{Bentz2013}.  The typical uncertainties are shown on
the lower right of each panel. (*fig:sample*)

</div>
<div id="div_fig3">

<img src="tmp_2209.14410/./fig3.png" alt="Fig4" width="100%"/>

**Figure 4. -** Ratio of BLR and dust continuum sizes as a function of (a) the AGN
luminosity and (b) the Eddington ratio.  The $p$-values of Spearman's rank
correlation coefficient are reported in the lower right corner of each panel for
RM- and OI-measured samples.  The notations are the same as in
Figure \ref{fig:sc}.  The error bars are the quadrature sum of the uncertainties
of $\Rb$ and $\Rd$.  Panel (c) displays the histograms of the ratios.  The dashed
lines indicate the averaged $\Rb/$\Rd$$ based on the fitting of the $\Rb$--$\Rd$ relations with $\beta$ fixed to unity (see also Equation (\ref{eq:rst})).
$\Rb/$\Rd$$ does not show a statistically significant correlation with the $\lopt$ nor the Eddington ratio.  The linear scale of $\Rb/$\Rd$$ is indicated on
the right. (*fig:rat*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

50  publications in the last 7 days.
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers